In [65]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

import pickle

from tmu.models.autoencoder.autoencoder import TMAutoEncoder
from src.lib.care import calculate_care_score

In [66]:
# Create folder figures if it does not exist
os.makedirs("figures", exist_ok=True)

In [67]:
def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def load_test_status_labels(farm, event_id):
    z = np.loadtxt(f"./data_test/z_{farm}_{event_id}.txt", dtype=np.uint32)
    z = np.array(z).astype(np.uint32)
    return z


def load_test_label(farm, event_id):
    event_info = pd.read_csv(f"../../../data/care_to_compare/Wind Farm {farm}/event_info.csv", delimiter=';')

    metadata = event_info[event_info['event_id'] == event_id]

    event_label = metadata["event_label"].values[0]

    return False if event_label == "anomaly" else True

In [68]:
def load_model(filename) -> TMAutoEncoder:
    with open(filename, "rb") as f:
        model = pickle.load(f)

    return model

In [69]:
def calculate_accuracy(labels, predictions):
    # Calculate the accuracy
    accuracy = np.sum(labels == predictions) / len(labels)

    return accuracy

In [70]:
def reconstruction_accuracy(X, pred):
    correct = np.sum(X == pred)
    accuracy = correct / len(X)
    return accuracy


def plot_mse(X, y, pred, name, threshold):
    # Compute MSE for each row
    mse_per_row = [reconstruction_accuracy(X[i], pred[i]) for i in range(len(X))]

    # Plot SNS plot of all MSE values
    plt.figure(figsize=(8, 4))
    sns.histplot(mse_per_row, bins=50, kde=True, color='b')

    # Add a threshold line
    plt.axvline(threshold, color='r', linestyle='--')

    # Save the plot
    plt.savefig(f"./figures/plot_reconstruction_acc_{name}.png")

    plt.close()


def plot_predictions(X, y, z, pred, p, name, threshold):
    x = np.arange(0, len(X))  # Time or index
    r = [reconstruction_accuracy(X[i], pred[i]) for i in range(len(X))]

    plt.figure(figsize=(12, 6))

    # Plot reconstruction accuracy (main plot)
    plt.plot(x, r, label="Reconstruction Accuracy", color='blue', alpha=0.7)

    # Plot a line at the threshold
    plt.axhline(y=threshold, color='red', linestyle='--', label="Threshold")

    y_mapped = np.where(y == 0, -0.2, -0.1)
    plt.plot(x, y_mapped, label="Actual Anomalies (y)", color='red', linestyle='-', linewidth=2)

    # Plot p (predicted anomalies) as a binary line at -0.2 (stacked below y)
    p_mapped = np.where(p == 0, -0.3, -0.4)
    plt.plot(x, p_mapped, label="Predicted Anomalies (p)", color='green', linestyle='-', linewidth=2)

    z_mapped = np.where(np.logical_or(z == 0, z == 2), -0.5, -0.6)
    plt.plot(x, z_mapped, label="Status Type Id (z)", color='orange', linestyle='-', linewidth=2)

    # Formatting
    plt.xlabel("Time")
    plt.ylabel("Reconstruction Accuracy")
    plt.title("Anomaly Detection Visualization")

    # Set y-limits to make space for stacked lines
    plt.ylim(-0.7, 1.1)

    # Add horizontal reference lines for y and p
    plt.axhline(y=-0.2, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    plt.axhline(y=-0.4, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    plt.axhline(y=-0.6, color='black', linestyle='dotted', linewidth=1, alpha=0.5)

    # Add grid and legend
    plt.legend()
    plt.grid(True)

    # Save the plot
    plt.savefig(f"./figures/plot_detections_{name}.png")

    plt.close()


def get_predictions(X, y, z, name):
    tm = load_model("models/latest_24.pkl")

    pred = tm.predict(X)

    threshold = 0.75

    # For each row in pred, if the MSE is greater than the threshold, then it is an anomaly
    losses = [calculate_accuracy(X[i], pred[i]) for i in range(len(X))]

    X_predictions = np.array([1 if losses[i] < threshold else 0 for i in range(len(X))])

    # Accuracy
    accuracy = calculate_accuracy(y, X_predictions)

    plot_mse(X, y, pred, name, threshold)
    plot_predictions(X, y, z, pred, X_predictions, name, threshold)

    return X_predictions, accuracy

In [71]:
test_datasets = [34, 7, 53, 27, 19, 77, 83, 52, 21, 2, 23, 87, 74, 86, 82]


#test_datasets = [68, 51]

def run_prediction(farm, dataset):
    X = load_test_dataset(farm, dataset)
    y = load_test_labels(farm, dataset)
    z = load_test_status_labels(farm, dataset)

    is_normal = load_test_label(farm, dataset)

    predictions, accuracy = get_predictions(X, y, z, f"{farm}_{dataset}")

    return X, y, z, is_normal, predictions, accuracy


In [72]:
# Create a dataframe with status_type_id;label;prediction

elements = []

for set in test_datasets:
    res = run_prediction("B", set)

    result_df = pd.DataFrame({
        'status_type_id': res[2],
        'label': res[1],
        'prediction': res[4]
    })

    print(f"Done with {set}. Accuracy: {res[5]}")

    elements.append({'dataset': set, 'normal': res[3], 'data': result_df, 'accuracy': res[5]})

Done with 34. Accuracy: 0.8272045824199137
Done with 7. Accuracy: 0.8971807628524047


KeyboardInterrupt: 

In [10]:
score = calculate_care_score(elements)
print(score)

0


In [11]:
a = 0.7310675113777656
b = 0.7391092017989689
c = 0.6428237030635012

print((a + b + c) / 3)

0.7043334720800786
